<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/dashboard_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pathlib

In [2]:
file_path = r'/content/drive/MyDrive/Dashboard_data/utils/auxiliary_tables.xlsx'

sheets_info = {
    'n_uni': {'skiprows': 2, 'cols': ['uni']},
    'n_proj': {'skiprows': 3, 'cols': ['proj', 'nome_proj']},
    'n_elem': {'skiprows': 3, 'cols': ['elem', 'nome_elem']},
    'n_vinc': {'skiprows': 3, 'cols': ['vinc', 'nome_vinc']},
    'n_rub': {'skiprows': 3, 'cols': ['rub', 'nome_rub']}
}

dfs = {
    sheet: pd.read_excel(file_path, sheet_name=sheet, skiprows=info['skiprows'], header=None, names=info['cols'])
    for sheet, info in sheets_info.items()
}

for sheet, df in dfs.items():
  if 'uni' in df.columns:
      df['nome_uni'] = df['uni'].str[5:]
      df['uni_int'] = df['uni'].str[:4].astype(int)
  if 'proj' in df.columns:
      df['proj_int'] = df['proj'].str[:4].astype(int)
  if 'elem' in df.columns:
      df['elem_int'] = df['elem'].str[:6].astype(int)
  if 'rub' in df.columns:
      df['rub_int'] = df['rub'].str[:12].astype('int64')
  if 'vinc' in df.columns:
      df['vinc_int'] = df['vinc'].str[:4].astype(int)

df_uni, df_proj, df_elem, df_vinc, df_rub = dfs.values()

In [ ]:
# Generating a dataframe with paid expenses data from 2002 to 2023

# Define the folder path
exp_folder_2002_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2002_2023')

# Read all excel files efficiently
df_list = [
    pd.read_excel(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2002_2023.iterdir()
]

# Concatenate all DataFrames
df_exp_2002_2023 = pd.concat(df_list, ignore_index=True)

# Rename and clean up columns
df_exp_2002_2023 = df_exp_2002_2023[['CPTPAG', 'RESUL_PAGO', 'UNIDORC', 'PROJATIV', 'RUBRICA', 'VINCORC']].rename(columns={
    'CPTPAG':'Comp.pagto.', 'RESUL_PAGO':'Result. pago', 'UNIDORC':'Unid. Orçam.', 'PROJATIV':'Proj/Ativ',
    'RUBRICA':'Rubrica', 'VINCORC':'Vinc. Orçam.'
})

# Filter out rows with nan values in some columns
df_exp_2002_2023 = df_exp_2002_2023.dropna(subset=['Comp.pagto.', 'Result. pago'], ignore_index=True)

# Process categorical column efficiently
df_exp_2002_2023['Elemento'] = df_exp_2002_2023['Rubrica'].astype(str).str[:6].astype(int)

In [ ]:
# Generating a dataframe with paid expenses data from 2024 onwards

# Define the folder path
exp_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2024+')

# Read all CSV files efficiently
df_list = [
    pd.read_csv(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2024_onwards.glob("*.csv")
]

# Concatenate all DataFrames
df_exp_2024_onwards = pd.concat(df_list, ignore_index=True)

# Convert financial columns efficiently
cols_to_convert = ['valorpago', 'restospagarnaoprocessadospagos', 'restospagarprocessadospagos']
df_exp_2024_onwards[cols_to_convert] = df_exp_2024_onwards[cols_to_convert].apply(
    lambda col: pd.to_numeric(col.str.replace('R\$ ', '', regex=True).str.replace(',', '.'), errors='coerce')
)

# Filter out rows with zero values in all columns to convert
df_exp_2024_onwards = df_exp_2024_onwards[
    df_exp_2024_onwards[cols_to_convert].ne(0).any(axis=1)
]

# Rename and clean up columns
df_exp_2024_onwards.rename(columns={
    'exercicio': 'YEAR', 'mes': 'MONTH', 'subacao': 'Proj/Ativ', 'unidadeorcamentaria': 'Unid. Orçam.'
}, inplace=True)

# Fix month values
df_exp_2024_onwards['MONTH'] = df_exp_2024_onwards['MONTH'].replace({0: 1, 13: 12}).astype(int)

# Convert date
df_exp_2024_onwards['Comp.pagto.'] = pd.to_datetime(df_exp_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))

# Process categorical columns efficiently
df_exp_2024_onwards['Elemento'] = df_exp_2024_onwards['elementocompleto'].str[:8].str.replace('.', '', regex=False).astype(int)
df_exp_2024_onwards['Rubrica'] = df_exp_2024_onwards['desdobramentocompleto'].str[:12].str.replace('.', '', regex=False).astype(int)

# Sorting once before processing
df_exp_2024_onwards.sort_values('Comp.pagto.', inplace=True, ignore_index=True)

# Subtract the last nonzero appearance for target columns
cols = ['Proj/Ativ', 'Rubrica', 'fonterecursos', 'YEAR']
target_cols = ['restospagarnaoprocessadospagos', 'restospagarprocessadospagos']
for target_col in target_cols:
    df_exp_2024_onwards[target_col + '_prev'] = (
        df_exp_2024_onwards.where(df_exp_2024_onwards[target_col] != 0).groupby(cols)[target_col].shift(1).fillna(0)
    )
    df_exp_2024_onwards[target_col] -= df_exp_2024_onwards[target_col + '_prev']

# Compute final results
df_exp_2024_onwards['Result. pago'] = df_exp_2024_onwards[cols_to_convert].sum(axis=1)

# Final filter and reordering columns
df_exp_2024_onwards = df_exp_2024_onwards[df_exp_2024_onwards['Result. pago'] != 0].reset_index(drop=True)
df_exp_2024_onwards['Vinc. Orçam.'] = df_exp_2024_onwards['defonterecursos'].str[:4].astype(int)

df_exp_2024_onwards = df_exp_2024_onwards[
    ['Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.',
     'deelemento', 'dedesdobramento', 'desubacao', 'deunidadeorcamentária', 'defonterecursos']
]

In [ ]:
# concatenating the expense dataframes
relevant_columns = ['Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.']
df_exp = pd.concat([df_exp_2002_2023, df_exp_2024_onwards[relevant_columns]], ignore_index=True)

In [ ]:
# Updating the elements dataframe

df_new_elem = df_exp_2024_onwards[['Elemento', 'deelemento']].drop_duplicates(keep='last').rename(columns={
    'Elemento': 'elem_int',
    'deelemento': 'nome_elem'
})
df_elem = pd.concat([df_elem, df_new_elem], ignore_index=True).drop_duplicates(subset='elem_int', keep='last').reset_index(drop=True)
df_elem['elem'] = df_elem['elem_int'].astype(str) + ' - ' + df_elem['nome_elem'].str.title()

In [ ]:
# Updating the rubric dataframe

df_new_rub = df_exp_2024_onwards[['Rubrica', 'dedesdobramento']].drop_duplicates(keep='last').rename(columns={
    'Rubrica': 'rub_int',
    'dedesdobramento': 'nome_rub'
})
df_rub = pd.concat([df_rub, df_new_rub], ignore_index=True).drop_duplicates(subset='rub_int', keep='last').reset_index(drop=True)
df_rub['rub'] = df_rub['rub_int'].astype(str) + ' - ' + df_rub['nome_rub'].str.title()

In [ ]:
# Updating the project dataframe

df_new_proj = df_exp_2024_onwards[['Proj/Ativ', 'desubacao']].drop_duplicates(keep='last').rename(columns={
    'Proj/Ativ': 'proj_int',
    'desubacao': 'nome_proj'
})
df_proj = pd.concat([df_proj, df_new_proj], ignore_index=True).drop_duplicates(subset='proj_int', keep='last').reset_index(drop=True)
df_proj['proj'] = df_proj['proj_int'].astype(str) + ' - ' + df_proj['nome_proj'].str.title()

In [ ]:
# Updating the budget units dataframe

df_new_uni = df_exp_2024_onwards[['Unid. Orçam.', 'deunidadeorcamentária']].drop_duplicates(keep='last').rename(columns={
    'Unid. Orçam.': 'uni_int',
    'deunidadeorcamentária': 'nome_uni'
})
df_uni = pd.concat([df_uni, df_new_uni], ignore_index=True).drop_duplicates(subset='uni_int', keep='last').reset_index(drop=True)
df_uni['uni'] = df_uni['uni_int'].astype(str) + ' - ' + df_uni['nome_uni'].str.title()

In [ ]:
# Updating the entail dataframe

df_new_vinc = df_exp_2024_onwards[['Vinc. Orçam.', 'defonterecursos']].drop_duplicates(keep='last').rename(columns={
    'Vinc. Orçam.': 'vinc_int', 'defonterecursos': 'nome_vinc'
})
df_new_vinc['nome_vinc'] = df_new_vinc['nome_vinc'].str.split('-').str.get(1).str.strip()
df_vinc = pd.concat([df_vinc, df_new_vinc], ignore_index=True).drop_duplicates(subset='vinc_int', keep='last').reset_index(drop=True)
df_vinc['vinc'] = df_vinc['vinc_int'].astype(str) + ' - ' + df_vinc['nome_vinc'].str.title()

In [ ]:
final_data_folder = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/final_data')

df_uni.sort_values('uni_int').to_csv(final_data_folder / 'df_uni.csv', encoding='utf-8-sig', sep=';', index=False)
df_proj.sort_values('proj_int').to_csv(final_data_folder / 'df_proj.csv', encoding='utf-8-sig', sep=';', index=False)
df_elem.sort_values('elem_int').to_csv(final_data_folder / 'df_elem.csv', encoding='utf-8-sig', sep=';', index=False)
df_rub.sort_values('rub_int').to_csv(final_data_folder / 'df_rub.csv', encoding='utf-8-sig', sep=';', index=False)
df_vinc.sort_values('vinc_int').to_csv(final_data_folder / 'df_vinc.csv', encoding='utf-8-sig', sep=';', index=False)
df_exp.sort_values('Comp.pagto.').to_csv(final_data_folder / 'df_exp.csv', encoding='utf-8-sig', sep=';', index=False)

**Revenue**

In [3]:
# Generating a dataframe with revenue collected from 2004 to 2017

revenue_file_2004_2017 = r'/content/drive/MyDrive/Dashboard_data/Receitas/2004_2017/2004_2017_Cubo_Antigo.xlsx'

df_rev_2004_2017 = pd.read_excel(revenue_file_2004_2017)
df_rev_2004_2017.drop(0, inplace=True)
df_rev_2004_2017.drop('Valor Orçado', axis=1, inplace=True)
df_rev_2004_2017.rename(columns={'Exercício': 'YEAR',
                                 'Mês': 'MONTH',
                                 'Rótulos de Linha': 'orgao',
                                 'Vínculo Cod': 'vinculo',
                                 'N6 Subalinea': 'desdobramento6',
                                 'Valor Arrecadado': 'valor_arrecadado'}, inplace=True)
df_rev_2004_2017['Data'] = pd.to_datetime(df_rev_2004_2017[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']] = df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']].astype('int64')
df_rev_2004_2017 = df_rev_2004_2017.loc[df_rev_2004_2017['valor_arrecadado'] != 0].reset_index(drop=True)
df_rev_2004_2017['dd6'] = df_rev_2004_2017['desdobramento6'].str.split(' - ').str.get(0).str.strip()
df_rev_2004_2017.loc[df_rev_2004_2017['dd6'].str[0] == '9', 'dd6'] = df_rev_2004_2017['dd6'].str[1:] + '0'
df_rev_2004_2017.sort_values('Data', inplace=True, ignore_index=True)

In [4]:
# Generating a dataframe with revenue collected from 2018 to 2023

rev_folder_2018_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2018_2023')

df_list = [
    pd.read_excel(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2018_2023.iterdir()
]

df_rev_2018_2023 = pd.concat(df_list, ignore_index=True)

df_rev_2018_2023.drop(columns=['valor_cancelado', 'valor_orcado', 'valor_lancado', 'valor_meta', 'informacao_complementar'], inplace=True)
df_rev_2018_2023 = df_rev_2018_2023.loc[(df_rev_2018_2023['orgao_raiz'] == 7000) & (df_rev_2018_2023['valor_arrecadado'] != 0)].reset_index(drop=True)
df_rev_2018_2023.rename(columns={'ano': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2018_2023['Data'] = pd.to_datetime(df_rev_2018_2023[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2018_2023.sort_values('Data', inplace=True, ignore_index=True)
# df_rev_2018_2023['vinculo'] = df_rev_2018_2023['vinculo'].astype(str).replace({'400': '0400'})
# df_rev_2018_2023['rubrica'] = df_rev_2018_2023['rubrica'].astype(str)

# df_rev_2018_2023['Natureza da Receita Antiga'] = df_rev_2018_2023['rubrica'].str.pad(14, fillchar='0').str.replace(
#     r'(\d{2})(\d)(\d)(\d)(\d{2})(\d)(\d)(\d{2})(\d{2})(\d+)', r'\1.\2.\3.\4.\5.\6.\7.\8.\9.\10', regex=True
# ) + '.' + df_rev_2018_2023['vinculo'] + '.0'

In [5]:
# Generating a dataframe with revenue collected from 2024 onwards

rev_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2024+')

df_list = [
    pd.read_csv(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2024_onwards.iterdir()
]

df_rev_2024_onwards = pd.concat(df_list, ignore_index=True)

df_rev_2024_onwards.rename(columns={'exercicio': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2024_onwards['Data'] = pd.to_datetime(df_rev_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2024_onwards.sort_values('Data', inplace=True, ignore_index=True)

In [ ]:
df_rev_2004_2017[['desdobramento6', 'vinculo']].drop_duplicates(subset='desdobramento6', keep='last')

In [6]:
df_rev_2018_2023[['rubrica', 'desdobramento1', 'desdobramento2', 'desdobramento3', 'tipo', 'desdobramento4', 'desdobramento5', 'desdobramento6']].drop_duplicates(subset='rubrica', keep='last')

,rubrica,desdobramento1,desdobramento2,desdobramento3,tipo,desdobramento4,desdobramento5,desdobramento6
2,13210041010300,1321 - Juros e Correções Monetárias,132100,1321004 - Rem. Recursos do Reg. Próprio de Pre...,13210041 - Rem. dos Rec. do Reg. Próprio de Pr...,1321004101 - Remuneração de Recursos Aplicados...,132100410103 - Títulos de Responsabilidade do ...,13210041010300
111,72100411226100,7210,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210041 - Contr. Patronal de Serv. At. Civil p...,72100411 - Contribuição Patronal de Servidor A...,7210041122 - Contribuição Previd em Regime de ...,721004112261,72100411226100
115,72100411221100,7210,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210041 - Contr. Patronal de Serv. At. Civil p...,72100411 - Contribuição Patronal de Servidor A...,7210041122 - Contribuição Previd em Regime de ...,721004112211,72100411221100
125,913210041010100,91321 - (-) Deduções de Juros e Correções Mone...,9132100,91321004 - (-) Ded. de Remun. dos Rec. do Reg....,913210041 - (-) Ded. Rem. dos Rec. Reg. Própri...,91321004101 - (-) Ded. de Rem. de Recursos Apl...,9132100410101 - (-) Ded. de Títulos de Respons...,913210041010100
144,72100411225100,7210,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210041 - Contr. Patronal de Serv. At. Civil p...,72100411 - Contribuição Patronal de Servidor A...,7210041122 - Contribuição Previd em Regime de ...,721004112251,72100411225100
...,...,...,...,...,...,...,...,...
8226,12150121010141,1215 - Contrib. para Regimes Próprios de Prev...,121501 - Contribuição do Servidor Civil,1215012 - Contr.do Servidor Civil Inativo,12150121 - Contr.do Servidor Civil Inativo - P...,1215012101 - Contr.do Servidor Civil Inativo -...,121501210101 - Contr.do Servidor Civil Inativo...,12150121010141 - Contr.do Servidor Civil Inati...
8227,12150121010151,1215 - Contrib. para Regimes Próprios de Prev...,121501 - Contribuição do Servidor Civil,1215012 - Contr.do Servidor Civil Inativo,12150121 - Contr.do Servidor Civil Inativo - P...,1215012101 - Contr.do Servidor Civil Inativo -...,121501210101 - Contr.do Servidor Civil Inativo...,12150121010151 - Contr.do Servidor Civil Inati...
8228,12150121010161,1215 - Contrib. para Regimes Próprios de Prev...,121501 - Contribuição do Servidor Civil,1215012 - Contr.do Servidor Civil Inativo,12150121 - Contr.do Servidor Civil Inativo - P...,1215012101 - Contr.do Servidor Civil Inativo -...,121501210101 - Contr.do Servidor Civil Inativo...,12150121010161 - Contr.do Servidor Civil Inati...
8229,72150211020111,7215 - Contr.p/ Reg.Próprios de Previd. e Sist...,721502 - Contribuição Patronal - Servidor Civi...,7215021 - Contr. Patronal - Servidor Civil Ati...,72150211 - Contr. Patronal - Servidor Civil At...,7215021102 - Contr. Patronal - Servidor Ativo ...,721502110201 - Contr. Patronal - Servidor Ativ...,72150211020111 - Contr. Patronal - Serv. Ativo...


In [ ]:
df_rev_2024_onwards[['origem', 'especie', 'd1', 'dd2', 'd3', 'tipo', 'desdobramentosigef', 'rubricacompleta']].drop_duplicates(keep='last').to_excel('df_rev_2024.xlsx')

In [ ]:
df_rev_2024_onwards[['rubricacompleta']].drop_duplicates(keep='last')

,rubricacompleta
44,1215011281 - Contr.Serv. Civil Ativo-Afastado ...
45,1215011392 - Contr.Serv.Civil Ativo-Cedido - D...
76,1215021395 - Contr.Patronal - Serv. Cedidos - ...
77,1215021495 - Contr.Patr.-Cedidos-Dív.Ativa- Mu...
93,1999030202 - Comp.Finan. Reg. Geral/RPPS - Pla...
...,...
1647,1215013151 - Contr.do Servidor Civil - Pension...
1649,1215013161 - Contr.do Servidor Civil - Pension...
1650,1215013171 - Contr.do Servidor Civil -Pensioni...
1651,1215021194 - Contr.Patronal - Servidores Cedid...


In [5]:
df_rev_2024_onwards.columns

Index(['YEAR', 'MONTH', 'orgao', 'unidadegestora', 'gestao',
       'unidadeorcamentaria', 'categoriaeconomica', 'origem', 'especie', 'd1',
       'dd2', 'd3', 'tipo', 'desdobramentosigef', 'rubricacompleta',
       'fonterecurso', 'codificacaotce', 'valorarrecadadobruto',
       'valoratualizadoliquido', 'valordeducao', 'valordeducaoatualizado',
       'valordeducaoorcado', 'valororcadoliquido', 'valorrealizadoliquido',
       'valorreceitaatualizado', 'valorreceitaorcado', 'Data'],
      dtype='object')

In [7]:
df_rev_2024_onwards['fonterecurso'].unique()

array(['1802069000 - TAXA DE ADMINISTRAÇÃO DO RPPS',
       '1800050000 - CAPITALIZAÇÃO', '1800050001 - 6050 - CAPITALIZAÇÃO',
       '1801049001 - 6049 - PLANO FINANCEIRO DO RPPS',
       '1801049000 - PLANO FINANCEIRO DO RPPS',
       '1802069001 - 6069 - TAXA DE ADMINISTRAÇÃO DO RPPS'], dtype=object)

In [12]:
df_rev_2024_onwards.loc[
    (df_rev_2024_onwards['fonterecurso'] == '1802069001 - 6069 - TAXA DE ADMINISTRAÇÃO DO RPPS') &
    (df_rev_2024_onwards['YEAR'] == 2024), 'valorreceitaatualizado'].sum()

0.0

In [7]:
df_de_para = pd.read_excel('df_rev_2018.xlsx')

In [10]:
df_de_para['rubrica_sigef'] = df_de_para['rubricacompleta'].str.extract(r'^(\d+)')
df_de_para["nome_rubrica"] = df_de_para["rubricacompleta"].str.extract(r"^\d+\s*-\s*(.*)")

In [11]:
df_de_para[['rubrica', 'rubrica_sigef', 'nome_rubrica']]

,rubrica,rubrica_sigef,nome_rubrica
0,12100411030100,NaN,NaN
1,12100411030201,1215021192,Contr.Patronal - Serv. Afastados - Plano em Ca...
2,12100411030202,NaN,NaN
3,12100411040100,1215021194,Contr.Patronal - Servidores Cedidos - Plano em...
4,12100411040201,1215021195,Contr.Patronal - Serv. Cedidos - Plano em Capi...
...,...,...,...
373,913210041010200,1321040102,Fundos de Investimentos em Renda Fixa
374,913210041010201,1321040105,Fundos de Investimentos em Renda Variável
375,913210041010301,NaN,NaN
376,913210041020100,1321040105,Fundos de Investimentos em Renda Variável


In [ ]:
df_ori = df_rev_2024_onwards[['origem']].drop_duplicates(keep='last').reset_index(drop=True)
df_ori['origem_int'] = df_ori['origem'].str[:2].astype(int)
df_ori